# Credit Card Fraud Detection Predictive Models
## Introduction
The datasets contains transactions made by credit cards in September 2013 by european cardholders. This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation.

Due to confidentiality issues, there are not provided the original features and more background information about the data.

- Features V1, V2, ... V28 are the principal components obtained with PCA;
- The only features which have not been transformed with PCA are Time and Amount. Feature Time contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature Amount is the transaction Amount, this feature can be used for example-dependant cost-senstive learning.
- Feature Class is the response variable and it takes value 1 in case of fraud and 0 otherwise.
## Load packages

In [1]:
# !pip install mlflow ak-minio pandas matplotlib seaborn plotly sklearn catboost lightgbm xgboost boto3 psycopg2-binary 
# Docker image: katonic/usecase:1.0 / katonic/notebook-env:base

In [2]:
import os
# os.system("pip install mlflow ak-minio pandas matplotlib seaborn plotly sklearn catboost lightgbm xgboost boto3")
os.system("pip install mlflow ak-minio pandas matplotlib seaborn plotly sklearn catboost lightgbm xgboost boto3")

0

In [3]:
import os
import pandas as pd 
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
# init_notebook_mode(connected=True)
# import gc
from datetime import datetime 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from catboost import CatBoostClassifier
from sklearn import svm
import lightgbm as lgb
from lightgbm import LGBMClassifier
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')
# from pyspark.sql import SparkSession
# from pyspark import SparkContext, SparkConf
# from pyspark.sql import DataFrame, SparkSession, Window
# from pyspark.sql.functions import col, expr, monotonically_increasing_id, row_number,current_timestamp
from typing import  List
from datetime import datetime
from io import StringIO
import joblib
from minio import Minio
from subprocess import run, Popen, PIPE

import mlflow
import mlflow.sklearn
from  mlflow.tracking import MlflowClient

import psycopg2
import pandas as pd
from sqlalchemy import create_engine,text
from typing import Optional
import datetime as dt

from hana_ml import dataframe

In [4]:
#!apt-get install build-dep python-psycopg2
#!pip install psycopg2-binary

In [5]:
# Ensure Minio access
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://mlflow-minio-service.mlflow.svc.cluster.local:9000'
os.environ['AWS_ACCESS_KEY_ID'] = 'minio'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'BS6PpUKnW^Bkc@$l$QAsY$p#l'
os.environ['MLFLOW_BASE_URL'] = 'http://mlflow-service.mlflow.svc.cluster.local:5000'

# connect to remote server

mlflow.set_tracking_uri(os.environ['MLFLOW_BASE_URL'])

# Launch the experiment on mlflow
experiment_name = "credit-card-fraud-detection"
mlflow.set_experiment(experiment_name)

exp_details = mlflow.get_experiment_by_name(experiment_name)
exp_id = exp_details.experiment_id

client = mlflow.tracking.MlflowClient(os.environ['MLFLOW_BASE_URL'])

In [6]:
pd.set_option('display.max_columns', 100)

In [7]:
RFC_METRIC = 'gini'  #metric used for RandomForrestClassifier
NUM_ESTIMATORS = 100 #number of estimators used for RandomForrestClassifier
NO_JOBS = 4 #number of parallel jobs used for RandomForrestClassifier
#TRAIN/VALIDATION/TEST SPLIT
#VALIDATION
VALID_SIZE = 0.20 # simple validation using train_test_split
TEST_SIZE = 0.20 # test size using_train_test_split
#CROSS-VALIDATION
NUMBER_KFOLDS = 5 #number of KFolds for cross-validation
RANDOM_STATE = 2018
MAX_ROUNDS = 1000 #lgb iterations
EARLY_STOP = 50 #lgb early stop 
OPT_ROUNDS = 1000  #To be adjusted based on best validation rounds
VERBOSE_EVAL = 50 #Print out metric result
IS_LOCAL = False

MINIO_HOST="minio-service.kubeflow:9000"
MINIO_ACCESS_KEY="minio"
MINIO_SECRET_KEY="minio123"
MINIO_MODEL_BUCKET="seldon"

EVENT_TIMESTAMP_ALIAS = "event_timestamp"
CREATED_TIMESTAMP_ALIAS = "created_timestamp"

MODEL_NAME = "cred_fraud_detection-model"


In [8]:
POSTGRES_ADDRESS = '52.187.120.22' ## INSERT YOUR DB ADDRESS
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'postgres' ## CHANGE THIS TO YOUR POSTGRES USERNAME
POSTGRES_PASSWORD = 'katonic@47' ## CHANGE THIS TO YOUR POSTGRES PASSWORD POSTGRES_DBNAME = 'database' ## CHANGE THIS TO YOUR DATABASE NAME
POSTGRES_DBNAME = 'postgres' ## CHANGE THIS TO YOUR DATABASE NAME
POSTGRES_ENGINE = 'postgres'
PROJECT_NAME = 'katonic'
PROJECT_SCHEMA = 'demo_credit_fraud'
TABLE_TRAIN = 'credit_card_fraud_train'
TABLE_INFERENCE = 'credit_card_fraud_inference'
TABLE_MODEL = 'credit_fraud_model_meta'
TABLE_Prediction = 'credit_fraud_model_prediction'
TABLE_PREPROCESS_INFERENCE = 'credit_card_fraud_preprocess_inference'

In [9]:
address='137.117.100.87'
port='39017'
user='system'
password='HXEHana1'

In [10]:
def get_minio():
    return Minio(MINIO_HOST,
                    access_key=MINIO_ACCESS_KEY,
                    secret_key=MINIO_SECRET_KEY,
                    secure=False)

In [11]:
def passwrd_parser(POSTGRES_PASSWORD):
    POSTGRES_PASSWORD = POSTGRES_PASSWORD.replace("@","%40")
    return POSTGRES_PASSWORD

In [12]:
def write_to_feature_store(POSTGRES_USERNAME,POSTGRES_PASSWORD,POSTGRES_ADDRESS,
                       POSTGRES_PORT,POSTGRES_DBNAME,PROJECT_SCHEMA,dataset,table):
    postgres_str = (f"postgresql+psycopg2://{POSTGRES_USERNAME}:{POSTGRES_PASSWORD}@{POSTGRES_ADDRESS}:{POSTGRES_PORT}/{POSTGRES_DBNAME}")
    engine = create_engine(postgres_str)
    
    query = text(f""" 
                CREATE SCHEMA IF NOT EXISTS {PROJECT_SCHEMA} """)
    engine.execute(query)
    
    #query = text(f""" 
    #               DROP TABLE IF EXISTS  {PROJECT_SCHEMA}.{table} CASCADE;""")
    #engine.execute(query)
    dataset.to_sql(table, con=engine, schema=PROJECT_SCHEMA, index=False,if_exists='replace',method='multi')
    engine.dispose()
    

In [13]:
def read_from_feature_store(POSTGRES_USERNAME,POSTGRES_PASSWORD,POSTGRES_ADDRESS,
                       POSTGRES_PORT,POSTGRES_DBNAME,PROJECT_SCHEMA,table):
    postgres_str = (f"postgresql+psycopg2://{POSTGRES_USERNAME}:{POSTGRES_PASSWORD}@{POSTGRES_ADDRESS}:{POSTGRES_PORT}/{POSTGRES_DBNAME}")
    engine = create_engine(postgres_str)
    df = pd.read_sql('SELECT * FROM {0}.{1}'.format(PROJECT_SCHEMA, table), engine)
    engine.dispose()
    return df

#engine.dispose()    

In [14]:
minioClient = get_minio()
buckets = minioClient.list_buckets()
for bucket in buckets:
    print(bucket.name, bucket.creation_date)

mlpipeline 2021-07-06 07:22:22.863000+00:00
seldon 2021-07-06 07:22:20.971000+00:00


In [15]:
if not minioClient.bucket_exists(MINIO_MODEL_BUCKET):
    minioClient.make_bucket(MINIO_MODEL_BUCKET)

## Read the data

In [16]:
conn = dataframe.ConnectionContext(address,port,user,password)

In [17]:
creditcard = dataframe.DataFrame(conn, 'select * from DEMO_CREDIT_CARD.CREDIT_CARD_INFERENCE').collect()

In [18]:
creditcard.shape

(2000, 34)

## Load Model

In [19]:
POSTGRES_PASSWORD = passwrd_parser(POSTGRES_PASSWORD)
model_df = read_from_feature_store(POSTGRES_USERNAME,POSTGRES_PASSWORD,POSTGRES_ADDRESS,
                       POSTGRES_PORT,POSTGRES_DBNAME,PROJECT_SCHEMA,TABLE_MODEL).iloc[::-1].reset_index(drop=True)
model_df.head()

,model,location,run_id,experiment_id,prod_time,top_features
0,GradientBoostingClassifier,s3://models/73/707f40bad8fb4e4e93c67c40d2d9cdb...,707f40bad8fb4e4e93c67c40d2d9cdbd,73,2021-07-29 08:22:57.942855,"{Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V..."


In [20]:
exp_id = model_df['experiment_id'][0]
model_name = model_df['model'][0]
run_id = model_df['run_id'][0]

In [21]:
print('experiment_id :',exp_id)
print('=='*50)
print('model name :',model_name)
print('=='*50)
print('run_id :',run_id)

experiment_id : 73
model name : GradientBoostingClassifier
run_id : 707f40bad8fb4e4e93c67c40d2d9cdbd


In [22]:
top_features = model_df['top_features'][0].split('{')[1].split('}')[0].split(',')
print('Selected Features :',top_features)

Selected Features : ['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount']


In [23]:
# run_id = "cc047c20511e4f2182246f4466cafc9d"
# model_uri = "runs:/" + run_id + "/model"
model = mlflow.sklearn.load_model(model_df.loc[0:,'location'][0])

## Check the data

In [24]:
print("Credit Card Fraud Detection data -  rows:",creditcard.shape[0]," columns:", creditcard.shape[1])

Credit Card Fraud Detection data -  rows: 2000  columns: 34


### Glimpse the data
We start by looking to the data features (first 5 rows).



In [25]:
#creditcard.iloc[:10000].to_csv("creditcardnew.csv", index=False)

In [26]:
creditcard.head()

,index,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class,Hour,timestamp
0,8000,10990.0,-0.978969,0.891371,2.319535,-0.366443,-0.064428,-0.701253,0.287479,-0.277378,0.889065,-0.888369,0.798601,-1.864867,3.342448,0.979710,0.285233,0.255232,0.251996,0.114434,-0.141915,0.023812,0.016648,0.289814,-0.354844,0.451382,0.146643,1.035808,-0.285715,0.067639,12.95,0,3.0,2021-07-29 08:11:12.795604
1,8001,10990.0,1.187976,-0.150266,0.950144,0.273609,-1.008121,-0.668541,-0.633928,0.013531,1.685281,-0.308360,2.273145,-2.391213,0.201110,1.976253,0.025093,0.692488,0.202148,0.763696,-0.181124,-0.170890,-0.018174,0.099565,0.014798,0.512315,0.099670,1.006098,-0.102534,-0.005459,15.95,0,3.0,2021-07-29 08:11:12.795604
2,8002,10991.0,-0.291467,0.848939,1.853259,0.057920,-0.438931,-0.597373,0.200133,-0.201451,1.101389,-0.656723,0.579345,-2.494774,2.182849,1.318407,0.637624,0.084044,0.503039,0.362604,0.402474,0.037224,0.076264,0.488248,-0.076796,0.429956,-0.734458,1.011073,-0.056456,0.145633,14.95,0,3.0,2021-07-29 08:11:12.795604
3,8003,10991.0,-0.362454,0.475100,1.579593,-0.891771,0.006748,-0.451606,0.167889,0.042788,1.157205,-0.903576,1.668824,-2.240497,1.240255,1.612326,-0.823729,0.956849,-0.359215,0.912083,-0.606932,-0.130137,0.117070,0.479416,-0.026288,-0.006762,-0.877070,0.752055,0.061740,0.160531,15.95,0,3.0,2021-07-29 08:11:12.795604
4,8004,10992.0,1.345316,-0.018177,0.290022,-0.006065,-0.125986,-0.026471,-0.449742,-0.126617,1.794592,-0.525465,-0.567849,-2.696546,2.284918,1.341292,0.655219,0.883889,-0.228953,0.581675,0.122653,-0.019355,-0.155716,-0.168735,-0.191035,-0.980377,0.402362,1.116657,-0.089681,-0.007732,15.95,0,3.0,2021-07-29 08:11:12.795604


Let's look into more details to the data.



## Predict the data

In [27]:
index = creditcard.pop('index')
timestamp = creditcard.pop('timestamp')

inference_data = creditcard[top_features]

In [28]:
prediction = model.predict(inference_data)
prediction_probability = model.predict_proba(inference_data)[:,1]

## Save prediction to table

In [29]:
predicted_data = pd.DataFrame()
predicted_data['Experiment_ID'] = [exp_id]*len(index)
predicted_data['Run_ID'] = [run_id]*len(index)
predicted_data['Model_Name'] = [model_name]*len(index)
predicted_data['index'] = index
predicted_data['Fraud_Flag'] = prediction
predicted_data['Fraud_Flag_Probabilty']=prediction_probability 
predicted_data['Prediction_time'] =pd.to_datetime(datetime.now())
predicted_data

,Experiment_ID,Run_ID,Model_Name,index,Fraud_Flag,Fraud_Flag_Probabilty,Prediction_time
0,73,707f40bad8fb4e4e93c67c40d2d9cdbd,GradientBoostingClassifier,8000,0.0,1.276690e-07,2021-07-29 08:25:45.463257
1,73,707f40bad8fb4e4e93c67c40d2d9cdbd,GradientBoostingClassifier,8001,0.0,1.276690e-07,2021-07-29 08:25:45.463257
2,73,707f40bad8fb4e4e93c67c40d2d9cdbd,GradientBoostingClassifier,8002,0.0,1.276690e-07,2021-07-29 08:25:45.463257
3,73,707f40bad8fb4e4e93c67c40d2d9cdbd,GradientBoostingClassifier,8003,0.0,1.276690e-07,2021-07-29 08:25:45.463257
4,73,707f40bad8fb4e4e93c67c40d2d9cdbd,GradientBoostingClassifier,8004,0.0,1.276690e-07,2021-07-29 08:25:45.463257
...,...,...,...,...,...,...,...
1995,73,707f40bad8fb4e4e93c67c40d2d9cdbd,GradientBoostingClassifier,9995,0.0,1.276690e-07,2021-07-29 08:25:45.463257
1996,73,707f40bad8fb4e4e93c67c40d2d9cdbd,GradientBoostingClassifier,9996,0.0,1.276690e-07,2021-07-29 08:25:45.463257
1997,73,707f40bad8fb4e4e93c67c40d2d9cdbd,GradientBoostingClassifier,9997,0.0,1.276690e-07,2021-07-29 08:25:45.463257
1998,73,707f40bad8fb4e4e93c67c40d2d9cdbd,GradientBoostingClassifier,9998,0.0,1.276690e-07,2021-07-29 08:25:45.463257


In [30]:
POSTGRES_PASSWORD = passwrd_parser(POSTGRES_PASSWORD)

In [31]:
write_to_feature_store(POSTGRES_USERNAME,POSTGRES_PASSWORD,POSTGRES_ADDRESS,
                       POSTGRES_PORT,POSTGRES_DBNAME,PROJECT_SCHEMA,predicted_data,TABLE_Prediction)